In [ ]:
# Import most generic modules
import importlib
import pathlib
import os
import sys
from datetime import datetime, timedelta
import pandas as pd
from IPython.display import display, Markdown
import warnings

warnings.filterwarnings("ignore")
module_path = os.path.abspath(os.path.join("../.."))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# Parameters that will be replaced when calling this notebook
ticker = "AAPL"

report_name = (
    f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_due_diligence_{ticker.upper()}"
)

In [ ]:
display(
    Markdown(
        f"# {ticker.upper()} - Due Diligence - {datetime.now().strftime('%Y/%m/%d %H:%M:%S')}"
    )
)

In [ ]:
base_path = os.path.abspath(os.path.join("."))
report_cache_dir = pathlib.Path(base_path, "stored")

if not os.path.isdir(report_cache_dir):
    print("Reports data directory not found.")
    print(f"Creating {report_cache_dir}")
    os.mkdir(report_cache_dir)
else:
    print(f"Found reports directory {report_cache_dir}")

## Candle View

In [ ]:
from gamestonk_terminal.stocks import stocks_helper

df_stock_cache = pathlib.Path(report_cache_dir, f"{ticker}_stock_data.pkl")
if os.path.isfile(df_stock_cache):
    print(f"Found a cache file. Loading {df_stock_cache}")
    df_stock = pd.read_pickle(df_stock_cache)
else:
    print("Cache file not found. Getting data.")
    df_stock = stocks_helper.load_ticker(
        ticker, (datetime.now() - timedelta(days=180)).strftime("%Y-%m-%d")
    )
    df_stock = stocks_helper.find_trendline(df_stock, "OC_High", "high")
    df_stock = stocks_helper.find_trendline(df_stock, "OC_Low", "low")
    print(f"Saving cache file {ticker}_stock_data.pkl")
    df_stock.to_pickle(df_stock_cache)

In [ ]:
import mplfinance as mpf

mc = mpf.make_marketcolors(
    up="green", down="red", edge="black", wick="black", volume="in", ohlc="i"
)
s = mpf.make_mpf_style(marketcolors=mc, gridstyle=":", y_on_right=True)

ap0 = []

if "OC_High_trend" in df_stock.columns:
    ap0.append(
        mpf.make_addplot(df_stock["OC_High_trend"], color="g"),
    )

if "OC_Low_trend" in df_stock.columns:
    ap0.append(
        mpf.make_addplot(df_stock["OC_Low_trend"], color="b"),
    )

mpf.plot(
    df_stock,
    type="candle",
    mav=(20, 50, 200),
    volume=True,
    addplot=ap0,
    xrotation=0,
    style=s,
    figratio=(10, 7),
    figscale=2.00,
    update_width_config=dict(
        candle_linewidth=1.0, candle_width=0.8, volume_linewidth=1.0
    ),
)

## Analyst Targets

In [ ]:
from gamestonk_terminal.stocks.due_diligence import finviz_view

try:
    finviz_view.analyst(ticker=ticker, export="")
except Exception as e:
    print(e)

## Fundamentals

### Annual Income Statement

In [ ]:
from gamestonk_terminal.stocks.fundamental_analysis import av_view

av_view.display_income_statement(ticker, limit=1)

### Quarterly Income Statement

In [ ]:
from gamestonk_terminal.stocks.fundamental_analysis import av_view

av_view.display_income_statement(ticker, limit=1, quarterly=True)

### Annual Balance Sheet

In [ ]:
from gamestonk_terminal.stocks.fundamental_analysis import av_view

av_view.display_balance_sheet(ticker, limit=1)

### Annual Cash Flow

In [ ]:
from gamestonk_terminal.stocks.fundamental_analysis import av_view

av_view.display_cash_flow(ticker, limit=1)

In [ ]:
!jupyter nbconvert {report_name + ".ipynb"} --to html --no-input